In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.metrics import f1_score

2023-05-30 13:56:17.674644: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/as274094/anaconda3/envs/psf/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
def SEDlisttoC(SED_list):
    sed_array = np.array(SED_list)
    return sed_array*0.5 + 1.5

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

def calculate_success_rate(confusion_matrix):
    diagonal = np.trace(confusion_matrix)
    diagonal_neighbors = np.sum(np.diagonal(confusion_matrix, offset=1)) + np.sum(np.diagonal(confusion_matrix, offset=-1))
    total_classified = np.sum(confusion_matrix)
    
    success_rate = (diagonal + diagonal_neighbors) / total_classified
    return success_rate

In [28]:
#Test classification metrics

example_SED_original = np.random.randint(low = 0, high = 13, size = 90)
example_C = SEDlisttoC(example_SED_original)
example_C_noisy = example_C + np.random.rand(example_C.shape[0])*0.8-0.40
example_SED_obtained = CtoSEDarray(example_C_noisy, np.zeros_like(example_C_noisy))
#for i in range(example_SED_original.shape[0]):
#    print(example_SED_original[i], example_C_noisy[i], example_SED_obtained[i])
confusion_matrix = tf.math.confusion_matrix(example_SED_original, example_SED_obtained)
confusion_matrix


<tf.Tensor: shape=(21, 21), dtype=int32, numpy=
array([[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [5, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 2, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 0, 0,

In [30]:
f1 = f1_score(example_SED_original, example_SED_obtained, average = None)

print("F1 Score:", f1)
print(np.mean(f1[:13]))

success_rate = calculate_success_rate(confusion_matrix)
print('Success rate:', success_rate)

F1 Score: [0.5        0.625      0.6        0.6        0.66666667 0.54545455
 0.53333333 0.66666667 0.61538462 0.6        0.44444444 0.66666667
 0.73684211 0.        ]
0.6000353110676996
Success rate: 0.9333333333333333


In [25]:
#F1 score with tensorflow addons (deprecated)

# One-hot the classes
SEDtrue_oh = tf.one_hot(example_SED_original, depth=21)
SEDpred_oh = tf.one_hot(example_SED_obtained, depth=21)

metric = tfa.metrics.F1Score(num_classes = 21,threshold=0.5)
metric.update_state(SEDtrue_oh, SEDpred_oh)
result = metric.result()
print(result.numpy())
print(tf.math.reduce_mean(result[:13]))

[0.46153846 0.70588243 0.61538464 0.53333336 0.33333334 0.5
 0.71428573 0.6        0.66666675 0.5        0.46153846 0.63157904
 0.6        0.         0.         0.         0.         0.
 0.         0.         0.        ]
tf.Tensor(0.56334937, shape=(), dtype=float32)
